In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier

import helper
import text_embeddings
import performance
import models
import predict

# 1. Load data

In [2]:
x, y = helper.load_data()

# 2. Split data into train/test

In [3]:
x_train, x_test, y_train, y_test = helper.split_data(x, y)

# 3. Text embedding

In [4]:
x_train_features, x_test_features, vectorizer = text_embeddings.encode_bag_of_words(x_train, x_test, 'text')

# 4. Grid Search using 

-  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
-  https://scikit-learn.org/stable/modules/kernel_approximation.html  

In [5]:
final_x_train = x_train_features['text']['data']
final_y_train = y_train

final_x_test = x_test_features['text']['data']
final_y_test = y_test

In [8]:
model_params = dict(eta0=0.0001,loss='hinge', random_state=15, penalty='l2', tol=1e-3, verbose=0)
model_class = SGDClassifier

In [ ]:
%%time
import matplotlib.pyplot as plt
# from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint


alpha_range = [ 0.0001,0.001,0.01,0.1,1,2,3,4,5,6,7,8,9,10,50,100]

model = model_class(**model_params)

search_params = {'alpha' : alpha_range}
search = GridSearchCV(model,
                      search_params,
                      cv=5,
                      scoring="f1",
                     return_train_score=True)


search.fit(final_x_train, final_y_train)
results = pd.DataFrame.from_dict(search.cv_results_)
results = results.sort_values(['param_alpha'])
results['mean_train_score-mean_test_score'] = results['mean_train_score'] - results['mean_test_score']
results[['param_alpha','mean_train_score','mean_test_score', 'mean_train_score-mean_test_score']]

In [ ]:
# Train
plt.plot(results['param_alpha'], results['mean_train_score'], label="Train AUC")
plt.scatter(results['param_alpha'], results['mean_train_score'], label="Train AUC points")

# Validation
plt.plot(results['param_alpha'], results['mean_test_score'], label="Validation AUC")
plt.scatter(results['param_alpha'], results['mean_test_score'], label="Validation AUC points")

plt.xlabel("Alpha: hyperparameter")
plt.ylabel("AUC")
plt.title("AUC vs alpha curves")
plt.legend()
plt.show()

# 5. Prediction

In [9]:
alpha = 0.01
final_model = model_class(alpha=alpha, **model_params)
final_model.fit(final_x_train, final_y_train)

final_y_train_pred = final_model.predict(final_x_train)
final_y_test_pred = final_model.predict(final_x_test)


# 6. Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Train
train_cm = confusion_matrix(final_y_train, final_y_train_pred)
train_cm = pd.DataFrame(train_cm, columns=['actual_0', 'actual_1'],
                            index=['predicted_0','predicted_1'])
print("Confusion matrix for Train set")
print(train_cm)
print()

# Test
test_cm = confusion_matrix(final_y_test, final_y_test_pred)
test_cm = pd.DataFrame(test_cm, columns=['actual_0', 'actual_1'],
                            index=['predicted_0','predicted_1'])
print("Confusion matrix for Test set")
print(test_cm)

# 7. Performance Measures

In [12]:
performance_measures = performance.get_performance_measures(final_model, final_x_test, final_y_test)
performance_measures

{'accuracy': 0.8959937018894332,
 'f1': 0.928849260965831,
 'confusion_matrix':              actual_0  actual_1
 predicted_0      2482       634
 predicted_1       555      7761}

# Test

In [26]:
text = "happy January"
predict.is_text_depressed(final_model, text, 
                         model_id="NB",
                         vectorizer=vectorizer)

0

# Future Work
- Explore string kernel https://github.com/timshenkao/StringKernelSVM
- word2vec using SVM (https://shop.tarjomeplus.com/UploadFileEn/TPLUS_EN_3959.pdf)
- https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/